In [1]:
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install pyngrok
!pip install twilio

In [1]:
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd "/content/gdrive/MyDrive/Colab Notebooks"
%run 'c01_proces_texto_en.ipynb'
%run 'c02_pipeline_roberta.ipynb'

In [4]:
from flask import Flask, render_template , request
from twilio.twiml.messaging_response import MessagingResponse

from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

In [5]:
ngrok.set_auth_token('2HpELRgwnBK1g2a5v8S6Krr7Nrb_4jLHMUpVm2djDvnSpqqUj')

In [6]:
app_transformers = Flask(__name__)
 
run_with_ngrok(app_transformers)

@app_transformers.route("/", methods=['POST'])
def respuesta_mensaje_wp():
    
    # Mensaje que se lee
    msg = request.form.get('Body') # Reading the message from the whatsapp
    print("msg-->",msg)
    
    # Respuesta del modelo
    resp = MessagingResponse()
    reply = resp.message()

    # Create reply
    if msg.lower() in ["hi", 'hola']:
       reply.body("Hello 😊 I am Atenea, your virtual assistant from Wikipedia. What topic would you like to consult?")

    else:

      if ('?' not in msg) and ('topic' not in msg):       
              
        texto = procesamiento_tema(msg)
        texto = pd.Series(texto)
        texto.to_csv('texto_procesado.txt')  

        reply.body("What would you like to know?")

      if '?' in msg: 

        texto = pd.read_csv('texto_procesado.txt')      
        texto = texto['0'][0] 
        
        respuesta = modelo_roberta_pipeline(msg, texto)
        respuesta = re.sub(' \(score.*','', respuesta)
        print(respuesta)
        reply.body(respuesta)
    
      if 'change the topic' in msg:       

        reply.body("Of course😊, What other topic would you like to consult?")

    return str(resp)
 
if __name__ == "__main__":	
    app_transformers.run()